In [30]:
import os
import pandas as pd

datapath = "../data"

In [31]:
dfs = pd.read_excel(os.path.join(datapath, "MycetomaMolecules_noheads.xlsx"), sheet_name=None)

/home/gturon/miniconda3/envs/chem/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I1243 is marked as a date but the serial value 29082710 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/gturon/miniconda3/envs/chem/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I1258 is marked as a date but the serial value 12743772 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/gturon/miniconda3/envs/chem/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I1265 is marked as a date but the serial value 19752928 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/gturon/miniconda3/envs/chem/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:223: UserWarning: Cell I1351 is marked as a date but the serial value 21903448 is outside the limits for dates. The cell will be treated as an

In [32]:
for k,v in dfs.items():
    print("Name: ", k)
    print("Length: ", v.shape)

Name:  Pathogen Box Full data
Length:  (400, 55)
Name:  Stasis Box Full data
Length:  (400, 45)
Name:  MyOS Compounds
Length:  (883, 87)
Name:  Pandemic Response Box
Length:  (400, 49)
Name:  MMVBoxes screened
Length:  (1600, 68)
Name:  KCGS2.0 Kinase library
Length:  (304, 22)
Name:  Oncode library
Length:  (5631, 45)
Name:  Oncode library2
Length:  (5631, 45)
Name:  Full Epichem Library
Length:  (927, 995)
Name:  Medchemexpress Fungal Box
Length:  (378, 21)


In [4]:
from rdkit import Chem
from standardiser import standardise
import numpy as np

def standardise_smiles(smiles):
    st_smiles = []
    for smi in smiles:
        if smi is None:
            st_smi = np.nan
            st_smiles += [st_smi]
            continue
        smi = str(smi)
        smi = smi.strip()
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            st_smi = np.nan
            st_smiles += [st_smi]
            continue
        try:
            std_mol = standardise.run(mol)
            st_smi = Chem.MolToSmiles(std_mol, canonical=True)
            st_smiles += [st_smi]
        except:
            st_smi = Chem.MolToSmiles(mol, canonical=True)
            st_smiles += [st_smi]
        if std_mol is None:
            st_smi = Chem.MolToSmiles(mol, canonical=True)
            st_smiles += [st_smi]
            continue
    return st_smiles

# Datasets to use:

According to the project [wiki](https://github.com/OpenSourceMycetoma/General-Start-Here/wiki/Screening-of-the-Pandemic-Response-Box), the first screening was as follows: _The PRB chemical collection consists of 201 antibacterial, 153 antiviral and 46 antifungal agents. All compounds were screened against M.mycetomatis, giving 46 active compounds shown below (negative numbers indicate full pathogen growth inhibition)._ Based on this, we will use the 25 uM % inhibition endpoint whenever possible with a cut-off of 20 (max reported activity: 17.7).

Dataset by dataset:
- Pathogen Box: all molecules screened in 100 uM ("100 gem"), a selection in 25uM. Consider the non-screened in 25 as negatives and the "25mean" with cut-off at 20 to discriminate actives/inactives. SMILES column "smiles"
- Stasis Box: no SMILES available, discard data
- MyOS Compounds: data available in column "25 μM mean", SMILES column "SMILES"
- Pandemic response box: data available in "25 uM mean". Use to check the description above matches. SMILES column "SMILES"
- MMV Box: summary of PRB, Pathogen Box and others. Use columns "Mean rowth 100 uM (%) (Mm)" and "Mean growth 25 uM (%) (Mm)" and compare results with previous boxes. SMILES column is "smiles"
- KCGS2.0 Kinase library: SMILES column is "SMILES string". Data seems from % growth inhibition at 10, 2 and 1 uM. Also lower numbers indicate more activity of the compound (green labelled: 18 and below). Relevant activity columns: "10 μM A", "10 μM B", "2 μM A", "2 μM B", "1 μM A", "1 μM B"
- Oncode library: Data expressed in metabolic activity at 2uM ("Madurella mycetomatis metabolic activity at 2µM (%)"), labelled as active (has follow up experiments) 9.1 and below (next active is 21.7, so 20 could still be the cut-off). SMILES column "Smiles"
- Oncode library2: same as previous. Check if duplicated
- Full Epichem Library: data not clear, unprocessable results
- Medchemexpress Fungal Box: concentration labelled at 10mM? Percentage inhibition labelled in yellow under "Average" column indicates actives below 20

In [12]:
df = dfs["Pathogen Box Full data"]
df = df[["smiles", "100 gem", "25mean"]]
df["st_smiles"] = standardise_smiles(df["smiles"].tolist())
print(len(df[df["25mean"].isna()]))
df["exp"] = df["25mean"].fillna(100)
df["bin"] = [1 if x < 20 else 0 for x in df["exp"].tolist()]
print("Pathogen box: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]]
df.to_csv(os.path.join(datapath, "processed", "pathogen_box.csv"), index=False)

252
Pathogen box:  total:  400  actives:  39


/tmp/ipykernel_653961/4007570850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["st_smiles"] = standardise_smiles(df["smiles"].tolist())
/tmp/ipykernel_653961/4007570850.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["exp"] = df["25mean"].fillna(100)
/tmp/ipykernel_653961/4007570850.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [13]:
df = dfs["Pandemic Response Box"]
df = df[["SMILES", "25 uM mean"]]
df["st_smiles"] = standardise_smiles(df["SMILES"].tolist())
print(len(df[df["st_smiles"].isna()]))
print(len(df[df["25 uM mean"].isna()]))
df["bin"] = [1 if x < 20 else 0 for x in df["25 uM mean"].tolist()]
print("Pathogen box: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]]
df.to_csv(os.path.join(datapath, "processed", "prb_box.csv"), index=False)

0
0
Pathogen box:  total:  400  actives:  45


/tmp/ipykernel_653961/1170263845.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["st_smiles"] = standardise_smiles(df["SMILES"].tolist())
/tmp/ipykernel_653961/1170263845.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["bin"] = [1 if x < 20 else 0 for x in df["25 uM mean"].tolist()]


In [18]:
df = dfs["MMVBoxes screened"]
df = df[["smiles", "Mean rowth 100 uM (%) (Mm)","Mean growth 25 uM (%) (Mm)"]]
print(len(df[df["smiles"].isna()]))
df = df[~df["smiles"].isna()]
df["st_smiles"] = standardise_smiles(df["smiles"].tolist())
print(len(df[df["st_smiles"].isna()]))
df = df[~df["st_smiles"].isna()]
print(len(df[df["Mean rowth 100 uM (%) (Mm)"].isna()]))
print(len(df[df["Mean growth 25 uM (%) (Mm)"].isna()]))  
df["exp"] = df["Mean growth 25 uM (%) (Mm)"].fillna(100)
df["bin"] = [1 if x < 20 else 0 for x in df["exp"].tolist()]
print("MMV boxes: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]] 
df.to_csv(os.path.join(datapath, "processed", "mmv_box.csv"), index=False)

401


[11:31:54] SMILES Parse Error: syntax error while parsing: [C@](O)(\C(=C\C=C\[C@H](C)C\C(\C)=C\C[C@@]([H])(O[C@@]1(CC[C@H](C)[C@@H]([R])O1)C2)C[C@]2([H])OC3=O)\CO4)([C@@]3([H])C=C(C)[C@H]5O)[C@]45[H]
[11:31:54] SMILES Parse Error: check for mistakes around position 75:
[11:31:54] 1(CC[C@H](C)[C@@H]([R])O1)C2)C[C@]2([H])O
[11:31:54] ~~~~~~~~~~~~~~~~~~~~^
[11:31:54] SMILES Parse Error: Failed parsing SMILES '[C@](O)(\C(=C\C=C\[C@H](C)C\C(\C)=C\C[C@@]([H])(O[C@@]1(CC[C@H](C)[C@@H]([R])O1)C2)C[C@]2([H])OC3=O)\CO4)([C@@]3([H])C=C(C)[C@H]5O)[C@]45[H]' for input: '[C@](O)(\C(=C\C=C\[C@H](C)C\C(\C)=C\C[C@@]([H])(O[C@@]1(CC[C@H](C)[C@@H]([R])O1)C2)C[C@]2([H])OC3=O)\CO4)([C@@]3([H])C=C(C)[C@H]5O)[C@]45[H]'
[11:31:54] SMILES Parse Error: syntax error while parsing: O[C@@](\C(=C\C=C\[C@@H]([C@@H](\C(\C)=C\C[C@@H](O[C@@]1(C=C[C@@H](C)[C@H]([C@H](C)[R])O1)C2)C[C@H]2OC3=O)O[C@@H]4O[C@@H](C)[C@H](O[C@H]5O[C@H](C)[C@@H](O)[C@H](OC)C5)[C@@H](OC)C4)C)\CO6)([C@H]3C=C(C)[C@H]7O)C67
[11:31:54] SMILES Parse 

2
0
253
MMV boxes:  total:  1197  actives:  141


In [26]:
# compare prb, pathogen and mmv boxes
df_prb = pd.read_csv(os.path.join(datapath, "processed", "prb_box.csv"))
df_pat = pd.read_csv(os.path.join(datapath, "processed", "pathogen_box.csv"))
df_mmv = pd.read_csv(os.path.join(datapath, "processed", "mmv_box.csv"))

print(len(df_mmv), len(set(df_mmv["st_smiles"].tolist())))

df_int_prb = df_mmv[df_mmv["st_smiles"].isin(df_prb["st_smiles"].tolist())]
print(len(df_prb["st_smiles"].tolist()))
print(len(df_int_prb))
active_prb = df_prb[df_prb["bin"]==1]["st_smiles"].tolist()
active_mmv_prb = df_int_prb[df_int_prb["bin"]==1]["st_smiles"].tolist()
print("PRB-MMV intersection actives: ", len(active_mmv_prb), "total actives PRB: ", len(active_prb))
print(set(active_mmv_prb)-(set(active_prb)))

df_int_pat = df_mmv[df_mmv["st_smiles"].isin(df_pat["st_smiles"].tolist())]
print(len(df_pat["st_smiles"].tolist()))
print(len(df_int_pat))
active_pat = df_pat[df_pat["bin"]==1]["st_smiles"].tolist()
active_mmv_pat = df_int_pat[df_int_pat["bin"]==1]["st_smiles"].tolist()
print("PRB-MMV intersection actives: ", len(active_mmv_pat), "total actives Pat: ", len(active_pat))
print(set(active_mmv_pat)-(set(active_pat)))

# To avoid confusion, we will keep only molecules not appearing in prb or pathogen box
df_mmv_filt = df_mmv[~df_mmv["st_smiles"].isin(df_prb["st_smiles"].tolist())]
df_mmv_filt = df_mmv_filt[~df_mmv_filt["st_smiles"].isin(df_pat["st_smiles"].tolist())]
print("Filtered MMV box: ", len(df_mmv_filt), " actives: ", len(df_mmv_filt[df_mmv_filt["bin"]==1]))
df_mmv_filt.to_csv(os.path.join(datapath, "processed", "mmv_box_subset.csv"), index=False)

1197 1174
400
417
PRB-MMV intersection actives:  47 total actives PRB:  45
set()
400
412
PRB-MMV intersection actives:  41 total actives Pat:  39
{'O[C@@H](c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)[C@H]1CCCCN1'}
Filtered MMV box:  381  actives:  53


In [29]:
df

,57,MYOS Codes,Other ID,Retest,aka,Series,Origin/Synthesised by,Shipped on,Arrived at EMC on:,Year Screened,...,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Melanisation,Unnamed: 84,Unnamed: 85,Unnamed: 86
0,2,MYOS_00001_00_01,HPD14_1,NaN,EPL-BS0495,series 1,NaN,NaN,NaN,2019.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,MYOS_00001_00_02,EPL-BS0495,NaN,"HPD14_1, EPL-BS0495",series 1,Hung,NaN,NaN,2018.0,...,0.002444,0.8981,0.000189,no,no,0.195,2.429,2.791,0.0749,no
2,4,MYOS_00002_00_01,EPL-BS0038,NaN,NaN,series 1,NaN,NaN,NaN,2019.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,MYOS_00003_00_01,EPL-BS0800,am,"HPD10_1, EPL-BS0800",series 1,NaN,NaN,NaN,2018.0,...,0.002108,0.012 *,0.000183,yes,no,0.062,2.365,1.361,0.1171,no
4,6,MYOS_00003_00_02,HPD10_1,NaN,EPL-BS0800,series 1,Hung,NaN,NaN,2019.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,892,MYOS_00874_00_01,UCL8.18,NaN,NaN,series 2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879,893,MYOS_00875_00_01,UCL8.19,NaN,NaN,series 2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
880,894,MYOS_00876_00_01,AJ1C,note - not purified or sent,NaN,series 2,Audrey Jacques - USyd undergrad,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
881,895,MYOS_00877_00_01,AJ2C,NaN,NaN,series 2,Audrey Jacques - USyd undergrad,2025-11-10 00:00:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df = dfs["MyOS Compounds"]
df = df[["SMILES", "100 μM mean","25 μM mean"]]
print("No smiles: ", len(df[df["SMILES"].isna()]))
df = df[~df["SMILES"].isna()]
df["st_smiles"] = standardise_smiles(df["SMILES"].tolist())
print("No standard smiles: ",len(df[df["st_smiles"].isna()]))
df = df[~df["st_smiles"].isna()]
print("100uM no values: ", len(df[df["100 μM mean"].isna()]))
print("25uM no values: ", len(df[df["25 μM mean"].isna()]))
df = df[~df["25 μM mean"].isna()]
df["bin"] = [1 if x < 20 else 0 for x in df["25 μM mean"].tolist()]
print("MyOS compounds: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]]
df.to_csv(os.path.join(datapath, "processed", "myos_cpds.csv"), index=False)

No smiles:  36


[11:44:19] SMILES Parse Error: syntax error while parsing: (RR)
[11:44:19] SMILES Parse Error: check for mistakes around position 1:
[11:44:19] (RR)
[11:44:19] ^
[11:44:19] SMILES Parse Error: Failed parsing SMILES '(RR)' for input: '(RR)'
[11:44:19] SMILES Parse Error: syntax error while parsing: (RS)
[11:44:19] SMILES Parse Error: check for mistakes around position 1:
[11:44:19] (RS)
[11:44:19] ^
[11:44:19] SMILES Parse Error: Failed parsing SMILES '(RS)' for input: '(RS)'
[11:44:20] SMILES Parse Error: syntax error while parsing: O=C1C(O)=C(N2CCN(C3=CC=C(N+=O)C=C3)CC2)C(C4=CC=CC=C41)=O
[11:44:20] SMILES Parse Error: check for mistakes around position 27:
[11:44:20] O)=C(N2CCN(C3=CC=C(N+=O)C=C3)CC2)C(C4=CC=
[11:44:20] ~~~~~~~~~~~~~~~~~~~~^
[11:44:20] SMILES Parse Error: Failed parsing SMILES 'O=C1C(O)=C(N2CCN(C3=CC=C(N+=O)C=C3)CC2)C(C4=CC=CC=C41)=O' for input: 'O=C1C(O)=C(N2CCN(C3=CC=C(N+=O)C=C3)CC2)C(C4=CC=CC=C41)=O'
[11:44:20] SMILES Parse Error: syntax error while parsing: [][Fe][

No standard smiles:  7
100uM no values:  140
25uM no values:  139
MyOS compounds:  total:  701  actives:  253


In [44]:
df = dfs["KCGS2.0 Kinase library"]
df = df[["SMILES string", "10 μM A", "10 μM B", "2 μM A", "2 μM B", "1 μM A", "1 μM B"]]
print("NO smiles: ", len(df[df["SMILES string"].isna()]))
df["st_smiles"] = standardise_smiles(df["SMILES string"].tolist())
print("No st smiles: ", len(df[df["st_smiles"].isna()]))
df = df[~df["st_smiles"].isna()]
df["10um_avg"] = df[["10 μM A", "10 μM B"]].mean(axis=1)
df["2um_avg"] = df[["2 μM A", "2 μM B"]].mean(axis=1)
df["1um_avg"] = df[["1 μM A", "1 μM B"]].mean(axis=1)
print(len(df[df["10um_avg"].isna()]), len(df[df["2um_avg"].isna()]), len(df[df["1um_avg"].isna()]))
# keep values from 10 uM as everything else is negative
df["bin"] = [1 if x < 20 else 0 for x in df["10um_avg"].tolist()]
print("KCGS2.0 Kinase library: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]]
df.to_csv(os.path.join(datapath, "processed", "kcgs2_kinase.csv"), index=False)

NO smiles:  0
No st smiles:  4
0 0 0
KCGS2.0 Kinase library:  total:  300  actives:  5


[11:49:21] SMILES Parse Error: syntax error while parsing: EMPTY
[11:49:21] SMILES Parse Error: check for mistakes around position 1:
[11:49:21] EMPTY
[11:49:21] ^
[11:49:21] SMILES Parse Error: Failed parsing SMILES 'EMPTY' for input: 'EMPTY'
[11:49:21] SMILES Parse Error: syntax error while parsing: EMPTY
[11:49:21] SMILES Parse Error: check for mistakes around position 1:
[11:49:21] EMPTY
[11:49:21] ^
[11:49:21] SMILES Parse Error: Failed parsing SMILES 'EMPTY' for input: 'EMPTY'
[11:49:21] SMILES Parse Error: syntax error while parsing: EMPTY
[11:49:21] SMILES Parse Error: check for mistakes around position 1:
[11:49:21] EMPTY
[11:49:21] ^
[11:49:21] SMILES Parse Error: Failed parsing SMILES 'EMPTY' for input: 'EMPTY'
[11:49:21] SMILES Parse Error: syntax error while parsing: EMPTY
[11:49:21] SMILES Parse Error: check for mistakes around position 1:
[11:49:21] EMPTY
[11:49:21] ^
[11:49:21] SMILES Parse Error: Failed parsing SMILES 'EMPTY' for input: 'EMPTY'
/tmp/ipykernel_653961/14

In [50]:
df = dfs["Oncode library"]
df_ = dfs["Oncode library2"]
check = set(df["Smiles"].tolist()) - set(df_["Smiles"].tolist())
print(len(check)) # we will use only the first file as they are duplicated
df = df[["Smiles", "Madurella mycetomatis metabolic activity at 2µM (%)"]]
print(len(df[df["Smiles"].isna()]))
df["st_smiles"] = standardise_smiles(df["Smiles"].tolist())
print(len(df[df["st_smiles"].isna()]))
df = df[~df["st_smiles"].isna()]
df["bin"] = [1 if x < 20 else 0 for x in df["Madurella mycetomatis metabolic activity at 2µM (%)"].tolist()]
print("Oncode library: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]]
df.to_csv(os.path.join(datapath, "processed", "oncode_lib.csv"), index=False)

0
0


[11:53:11] SMILES Parse Error: syntax error while parsing: #N/B
[11:53:11] SMILES Parse Error: check for mistakes around position 1:
[11:53:11] #N/B
[11:53:11] ^
[11:53:11] SMILES Parse Error: Failed parsing SMILES '#N/B' for input: '#N/B'
[11:53:11] SMILES Parse Error: syntax error while parsing: OC[C@H]1O[C@@H]2O[C@H]3[C@H](O)[C@@H](O)[C@H](O[C@@H]3CO)O[C@H]4[C@H](O)[C@@H](O)[C@H](O[C@@H]4CO)O[C@H]5[C@H](O)[C@@H](O)[C@H](O[C@@H]5CO)O[C@H]6[C@H](O)[C@@H](O)[C@H](O[C@@H]6CO)O[C@H]7[C@H](O)[C@@H
[11:53:11] SMILES Parse Error: check for mistakes around position 200:
[11:53:11] )O[C@H]7[C@H](O)[C@@H
[11:53:11] ~~~~~~~~~~~~~~~~~~~~^
[11:53:11] SMILES Parse Error: Failed parsing SMILES 'OC[C@H]1O[C@@H]2O[C@H]3[C@H](O)[C@@H](O)[C@H](O[C@@H]3CO)O[C@H]4[C@H](O)[C@@H](O)[C@H](O[C@@H]4CO)O[C@H]5[C@H](O)[C@@H](O)[C@H](O[C@@H]5CO)O[C@H]6[C@H](O)[C@@H](O)[C@H](O[C@@H]6CO)O[C@H]7[C@H](O)[C@@H' for input: 'OC[C@H]1O[C@@H]2O[C@H]3[C@H](O)[C@@H](O)[C@H](O[C@@H]3CO)O[C@H]4[C@H](O)[C@@H](O)[C@H](O[C@@H]4

129
Oncode library:  total:  5502  actives:  31


[11:53:18] SMILES Parse Error: syntax error while parsing: #N/B
[11:53:18] SMILES Parse Error: check for mistakes around position 1:
[11:53:18] #N/B
[11:53:18] ^
[11:53:18] SMILES Parse Error: Failed parsing SMILES '#N/B' for input: '#N/B'
/tmp/ipykernel_653961/3083360058.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["st_smiles"] = standardise_smiles(df["Smiles"].tolist())


In [ ]:
- KCGS2.0 Kinase library: SMILES column is "SMILES string". Data seems from % growth inhibition at 10, 2 and 1 uM. Also lower numbers indicate more activity of the compound (green labelled: 18 and below). Relevant activity columns: "10 μM A", "10 μM B", "2 μM A", "2 μM B", "1 μM A", "1 μM B"
- Oncode library: Data expressed in metabolic activity at 2uM ("Madurella mycetomatis metabolic activity at 2µM (%)"), labelled as active (has follow up experiments) 9.1 and below (next active is 21.7, so 20 could still be the cut-off). SMILES column "Smiles"
- Oncode library2: same as previous. Check if duplicated
- Full Epichem Library: data not clear, unprocessable results
- Medchemexpress Fungal Box: concentration labelled at 10mM? Percentage inhibition labelled in yellow under "Average" column indicates actives below 20

In [59]:
df =dfs["Medchemexpress Fungal Box"]
df = df[["SMILES", "Average"]]
print(len(df[df["SMILES"].isna()]))
df["st_smiles"] = standardise_smiles(df["SMILES"].tolist())
print(len(df[df["st_smiles"].isna()]))
df = df[~df["st_smiles"].isna()]
df["bin"] = [1 if x < 20 else 0 for x in df["Average"].tolist()]
print("Medchemexpress Fungal Box: ", "total: ", len(df), " actives: ", len(df[df["bin"]==1]))
df = df[["st_smiles", "bin"]]
df.to_csv(os.path.join(datapath, "processed", "medchem_fungal_box.csv"), index=False)

0


[11:57:28] SMILES Parse Error: syntax error while parsing: [Saponins]
[11:57:28] SMILES Parse Error: check for mistakes around position 3:
[11:57:28] [Saponins]
[11:57:28] ~~^
[11:57:28] SMILES Parse Error: Failed parsing SMILES '[Saponins]' for input: '[Saponins]'
[11:57:28] SMILES Parse Error: syntax error while parsing: OC[C@@H](C(N(O)CCC(CCCCCCC(CCCC(C)C)=O)=O)=O)N.OS(O)(=O)=O.[1/2]
[11:57:28] SMILES Parse Error: check for mistakes around position 62:
[11:57:28] )=O)N.OS(O)(=O)=O.[1/2]
[11:57:28] ~~~~~~~~~~~~~~~~~~~~^
[11:57:28] SMILES Parse Error: Failed parsing SMILES 'OC[C@@H](C(N(O)CCC(CCCCCCC(CCCC(C)C)=O)=O)=O)N.OS(O)(=O)=O.[1/2]' for input: 'OC[C@@H](C(N(O)CCC(CCCCCCC(CCCC(C)C)=O)=O)=O)N.OS(O)(=O)=O.[1/2]'
[11:57:28] SMILES Parse Error: syntax error while parsing: [Tyrothricin]
[11:57:28] SMILES Parse Error: check for mistakes around position 2:
[11:57:28] [Tyrothricin]
[11:57:28] ~^
[11:57:28] SMILES Parse Error: Failed parsing SMILES '[Tyrothricin]' for input: '[Tyrothricin

5
Medchemexpress Fungal Box:  total:  373  actives:  72


[11:57:29] SMILES Parse Error: syntax error while parsing: CC(O)(C)C(Cl)(Cl)Cl.O.[1/2]
[11:57:29] SMILES Parse Error: check for mistakes around position 25:
[11:57:29] )(C)C(Cl)(Cl)Cl.O.[1/2]
[11:57:29] ~~~~~~~~~~~~~~~~~~~~^
[11:57:29] SMILES Parse Error: Failed parsing SMILES 'CC(O)(C)C(Cl)(Cl)Cl.O.[1/2]' for input: 'CC(O)(C)C(Cl)(Cl)Cl.O.[1/2]'
[11:57:29] SMILES Parse Error: syntax error while parsing: O[C@@H](C([O-])=O)[C@@H](O)[C@H](O)[C@H](O)CO.[1/2
[11:57:29] SMILES Parse Error: check for mistakes around position 49:
[11:57:29] C@H](O)[C@H](O)CO.[1/2
[11:57:29] ~~~~~~~~~~~~~~~~~~~~^
[11:57:29] SMILES Parse Error: Failed parsing SMILES 'O[C@@H](C([O-])=O)[C@@H](O)[C@H](O)[C@H](O)CO.[1/2' for input: 'O[C@@H](C([O-])=O)[C@@H](O)[C@H](O)[C@H](O)CO.[1/2'
/tmp/ipykernel_653961/46946334.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [71]:
# Join all binarised datasets
processed = ["pathogen_box", "prb_box", "mmv_box_subset", "myos_cpds", "kcgs2_kinase", "oncode_lib", "medchem_fungal_box"]

dfs = []
for p in processed:
    df = pd.read_csv(os.path.join(datapath, "processed", f"{p}.csv"))
    dfs += [df]
df = pd.concat(dfs, axis=0)
print("Final dataset: ", len(df), " actives: ", len(df[df["bin"]==1]))
print("Unique molecules: ", len(set(df["st_smiles"].tolist())))



Final dataset:  8057  actives:  498
Unique molecules:  7616


In [72]:
inconsistent = (
    df.groupby("st_smiles")["bin"]
      .nunique()
      .reset_index()
      .query("bin > 1")
)
print("Molecules with conflicting bin values: ", len(inconsistent))
print(len(inconsistent[inconsistent["bin"]>2]))

Molecules with conflicting bin values:  59
0


In [73]:
#all molecules duplicated have either same or two (0 and 1 values), but not three
# keep always active
df = (
    df.groupby("st_smiles")["bin"]
      .agg(lambda s: 1 if s.mean() >= 0.5 else 0)
      .reset_index()
)
print("total mols: ", len(df), "Actives: ", len(df[df["bin"]==1]))
df.to_csv(os.path.join(datapath, "ml_datasets", "final_dataset.csv"), index=False)

total mols:  7616 Actives:  445
